In [1]:
# import required packages
import requests
import ast # abstract syntax tree
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# get brewery names and locations

# connect to URL
URL= 'https://www.brewbound.com/breweries'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

# get rquired html - list of breweries is nested within map script
data = soup.find_all('script')[-4]
string_results = str(data).split("\r\n")[2]
string_results_list = "["+string_results[:len(string_results) - 7]+"]"
results = ast.literal_eval(string_results_list)

# convert list of lists to df
df_brewery = pd.DataFrame(results, columns = ['brewery_name', 'latitude','longitude','url'])
df_brewery.reset_index(inplace=True)
df_brewery.head()

# save as csv
# df_brewery.to_csv('breweries_2022.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5539 entries, 0 to 5538
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         5539 non-null   int64  
 1   brewery_name  5539 non-null   object 
 2   latitude      5539 non-null   float64
 3   longitude     5539 non-null   float64
 4   url           5539 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 216.5+ KB


In [24]:
# use brewery url to get beers

# make sure beer url is not cut short with ...
pd.set_option('display.max_colwidth', None)

# initialize string for results
beers = []

# iterate through all rows of df
for i in range(len(df_brewery['url'])):
# for i in range(10): # used for testing
    # get url
    url = df_brewery['url'].iloc[[i]].to_string(header=False, index=False)
    # print(i) # use if desired to monitor progress
    full_url = "https://www.brewbound.com"+url
    # connect to url - try statement used to avoid connection errors/ missing pages
    try:
        page = requests.get(full_url, allow_redirects=False)
        soup = BeautifulSoup(page.content, "html.parser")
        # find brewery location
        addresses = soup.find_all("address")
        # find beers table
        results = soup.find(id="beers")
    except:
        results = None
    
    # get address/ address for breweries with more than one location
    addr = []
    for address in addresses:
        addr.append(address.text.strip())

    # get beer table headers and add other required headers
    headers = []
    try:
        for j in results.find_all("th"):
            title = j.text
            headers.append(title)
        result_headers = headers[0:2]
        result_headers.append('beer_url')
        result_headers.append('beer_url2')
        result_headers.append('url')
        result_headers.append('location')
        
    except AttributeError:
        continue
    
    # get contents of beer table
    data = []
    
    try:
        rows = results.find_all('tr')
        for row in rows:
            # get beer name, style and url
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            # print(cols)
            clean_cols = []
            # Get rid of extra values but keep empty style
            for i in range(len(cols)):
                if i == 0 and cols[i] == "":
                    continue
                elif cols[i] == "":
                    if cols[i-1] != "":
                        clean_cols.append(cols[i])
                    else:
                        continue
                else:
                    clean_cols.append(cols[i])
            # if beer has no name but style add empty name
            if len(clean_cols) == 1:
                clean_cols.insert(0,"")
            print(clean_cols)
            # Remove empty strings
            cols = [ele for ele in clean_cols if ele != "No records to display."]
            urls = row.find_all('a', href=True)
            urls = [url['href'] for url in urls]
            all_cols = cols + urls
            data.append(all_cols)

        # add url and address to data - remove empty lists
        table = list(filter(None, data))
        table = [x.append(url) or x for x in table]
        table = [x.append(addr) or x for x in table]
        beers= beers + table
        
    except AttributeError:
        continue

# convert list of lists to df
df_beers = pd.DataFrame(beers, columns = result_headers)
df_beers = df_beers.drop(columns = 'beer_url2')
df_beers.head(15)
# save as csv
# df_beers.to_csv('beers_2022.csv')

[]
['Drumroll APA', 'American IPA']
['St. Lupulin', 'Pale Ale']
['The Meddler', 'Ale']
['Friek', 'Unknown']
['Isolation Ale', 'Ale']
['5 Barrel Pale Ale', 'Pale Ale']
['2015 Wolf Picker', 'Pale Ale']
['Fifty Niner', 'Golden or Blonde Ale']
['Perle White', 'White IPA']
['Jaunt', 'Ale']
['Footprint', 'Ale']
['Wolf Picker', 'Pale Ale']
['Runoff', 'IPA']
['Lugene', 'Stout']
['Woodcut No.07', 'Imperial Stout']
['Wellspring', 'Saison']
['Mountain Standard - Double Black India Pale Ale', 'Pale Ale']
['Fernet Aged Porter', 'Porter']
['Loose Leaf', 'Ale']
['Amuste', 'Porter']
['Woodcut No. 06', 'Ale']
['Woodcut No. 06', 'Ale']
['DeConstruction Golden Ale', 'Ale']
['Shenanigans', 'Ale']
['Pond Hopper', 'Pale Ale']
['Bourbon Barrel Stout 2011', 'Imperial Stout']
['Bourbon Barrel Stout', 'Imperial Stout']
['Double Pilsner', 'Imperial Pilsner']
['', 'Imperial Pilsner']
['', 'Imperial Pilsner']
['Myrcenary', 'Imperial IPA']
['Mountain Standard Double Black IPA', 'Black IPA']
['Saboteur', 'Brown Ale'

,Beer Name,Style,beer_url,url,location
0,Drumroll APA,American IPA,/breweries/Odell_Brewing_Co/drumroll-apa,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
1,St. Lupulin,Pale Ale,/breweries/Odell_Brewing_Co/St_Lupulin,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
2,The Meddler,Ale,/breweries/Odell_Brewing_Co/The_Meddler,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
3,Friek,Unknown,/breweries/Odell_Brewing_Co/Friek,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
4,Isolation Ale,Ale,/breweries/Odell_Brewing_Co/Isolation_Ale,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
5,5 Barrel Pale Ale,Pale Ale,/breweries/Odell_Brewing_Co/5_Barrel_Pale_Ale,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
6,2015 Wolf Picker,Pale Ale,/breweries/Odell_Brewing_Co/Roots_Release_2015_Wolf_Picker,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
7,Fifty Niner,Golden or Blonde Ale,/breweries/Odell_Brewing_Co/Fifty_Niner,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
8,Perle White,White IPA,/breweries/Odell_Brewing_Co/Roots_Release_Perle_White,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"
9,Jaunt,Ale,/breweries/Odell_Brewing_Co/Jaunt,/breweries/Odell_Brewing_Co/,"[Fort Collins, CO]"


In [42]:
# use beer url to get more beer info

# make sure beer url is not cut short with ...
pd.set_option('display.max_colwidth', None)

# df_beers= pd.read_csv('beers_2022.csv') # use if running scrapes separately

beers_info = []

# iterate through beer_urls
for i in range(len(df_beers['beer_url'])):
# for i in range(500,1000): # used for testing
    url = df_beers['beer_url'].iloc[[i]].to_string(header=False, index=False)
    # print(i) # use if desired to monitor progress
    beer_url =  "https://www.brewbound.com"+url
    try:
        # get html that contains description/ABV/IBU info
        page = requests.get(beer_url)
        soup = BeautifulSoup(page.content, "html.parser")
        rows = soup.find_all('div', {"class": "col-sm-12", "id": "brandtext"})
    except:
        rows = None

    info = []
    headers = []
    
    try:
        for row in rows:
            # get column headers
            strong = row.find_all("strong")
            strong = [x.text for x in strong]
            row = row.text
            row = row.split("\n")
            row = [x.strip() for x in row]
            headers.append(strong)
            info.append(row)

        # info[0] contains description, info[1] contains availability, ibu, abv etc.
        # join all sentences in description to single string
        description = info[0]
        description.remove('Official description from the brewery:')
        description = ' '.join(description).strip()
        info = [description] +info[1]
        
        # flatten headers into single list
        headers = [item for sublist in headers for item in sublist]

        # remove empty strings unless correspond to missing data  
        clean_info = []
        for i in range(len(info)):
            if info[i] =='':
                if info[i-1] in headers:
                    clean_info.append(info[i])
                else:
                    continue
            else:
                clean_info.append(info[i])

        # remove headers from list and add url to data so join possible
        results = [x for x in clean_info if x not in headers]
        results.append(url)
        
        beers_info.append(results)
    except:
        continue

# add url column header
headers.append('beer_url')

# convert list of lists to dataframe
df_beers_info = pd.DataFrame(beers_info, columns= headers)
df_beers_info.head()

# save as csv
# df_beers_info.to_csv('beersinfo_2022.csv')

,Official description from the brewery:,Beer Style,Availability,ABV,IBU,beer_url
0,No description for this beer,Pale Ale,Year Round,,,/breweries/Defiant_Brewing_Co/Defiant_Brewing_Co_Very_Hoppy_Pale_Ale
1,No description for this beer,Ale,Year Round,5.2,,/breweries/Defiant_Brewing_Co/Defiant_Brewing_Co_The_Horsemans_Ale
2,No description for this beer,IPA,Year Round,,,/breweries/Defiant_Brewing_Co/Defiant_Brewing_Co_Single_Finger_IPA
3,No description for this beer,Belgian,Year Round,,,/breweries/Defiant_Brewing_Co/Defiant_Brewing_Co_Resolution
4,No description for this beer,Lager,Year Round,,,/breweries/Defiant_Brewing_Co/Defiant_Brewing_Co_Prohibition_Lager


In [3]:
# Dictionary to map state to abbreviation in clean up below
# from https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [16]:
# Clean and join datasets
df_beers= pd.read_csv('beers_2022.csv') # use this if running each scrape separately
df_beers_info = pd.read_csv('beersinfo_2022.csv') # use this if running each scrape separately

# Clean up location string and split to city and state/country
df_beers['location_flat'] = df_beers['location'].astype(str)
df_beers['location_flat'] = df_beers['location_flat'].dropna().apply(lambda x: x.replace("\'","").replace("[","").replace("]","").replace("\"","").replace("\n"," "))
df_beers[['city', 'state_country']] = df_beers['location_flat'].str.split(',',n=1, expand=True)
df_beers['city'] = df_beers['city'].dropna().apply(lambda x: x.strip())
df_beers['state_country'] = df_beers['state_country'].dropna().apply(lambda x: x.replace("USA","").replace("U.S.A.","").replace("U.S.A","")
                            .replace("United States","").replace("Ontario","").replace(",","").replace("Orange County","CA").replace("Maui","").replace("West","")
                            .replace("FRANCE","France").replace("Island","OH"))
df_beers['state_country'] = df_beers['state_country'].dropna().apply(lambda x: x.strip())
df_beers['state_country'] = df_beers['state_country'].map(us_state_to_abbrev).fillna(df_beers['state_country'])
df_beers_info = df_beers_info.rename(columns={"Official description from the brewery:": "description"})
df_beers_final = pd.merge(df_beers,df_beers_info[['description','Availability','ABV','IBU','beer_url']],on='beer_url',how='left')

df_brewery_final = df_brewery.rename(columns={"index":"brewery_id"})
df_brewery_final = pd.merge(df_brewery_final, df_beers_final[['url','state_country','city']], on='url', how='left').drop_duplicates()

# add brewery id to beers df
df_beers_final = pd.merge(df_beers_final,df_brewery_final[['url','brewery_id']],on='url',how='left')

# drop url from brewery
df_brewery_final = df_brewery_final.drop(columns=['url'])

df_beers_final = df_beers_final.rename(columns={"Unnamed: 0": "beer_id"})
df_beers_final = df_beers_final.drop(columns = ['beer_url','url','location','location_flat','city','state_country'])

print(df_brewery.info())
print(df_beers_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5539 entries, 0 to 5538
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         5539 non-null   int64  
 1   brewery_name  5539 non-null   object 
 2   latitude      5539 non-null   float64
 3   longitude     5539 non-null   float64
 4   url           5539 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 216.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14241 entries, 0 to 14240
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   beer_id       14241 non-null  int64  
 1   Beer Name     14241 non-null  object 
 2   Style         14221 non-null  object 
 3   description   12862 non-null  object 
 4   Availability  12862 non-null  object 
 5   ABV           8960 non-null   object 
 6   IBU           4805 non-null   object 
 7   brewery_id    14230 non-

In [17]:
# given only 26% of breweries gave beer information - and thus city/state. Decided to try reverse geocoding with geopy
import time
from geopy.geocoders import Nominatim
  
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

city = []
state = []
country = []

for i in range(df_brewery.shape[0]):
# for i in range(12): # used for testing
    # get latitude and longitude as string
    Latitude = df_brewery['latitude'].iloc[[i]].to_string(header=False, index=False)
    Longitude = df_brewery['longitude'].iloc[[i]].to_string(header=False, index=False)

    # Nominatime api will only permit one API request per second so sleep timer used to ensure rate does not exceed limit
    time.sleep(1)
    try:
        # reverse geocode 
        location = geolocator.reverse(Latitude+","+Longitude)
        address = location.raw['address']
    
        # extract state and country
        city_i = address.get('city', '')
        city.append(city_i)
        state_i = address.get('state', '')
        state.append(state_i)
        country_i = address.get('country', '')
        country.append(country_i)
    except:
        # if request fails add empty string to list
        city.append("")
        state.append("")
        country.append("")

# add data to brewery data frame
df_brewery['state_geo']= state
df_brewery['country_geo']= country
df_brewery['city_geo'] = city

# check for nulls in reverse geocoded columns
df_brewery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5539 entries, 0 to 5538
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         5539 non-null   int64  
 1   brewery_name  5539 non-null   object 
 2   latitude      5539 non-null   float64
 3   longitude     5539 non-null   float64
 4   url           5539 non-null   object 
 5   state_geo     5539 non-null   object 
 6   country_geo   5539 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 303.0+ KB


In [18]:
# save dataframes as sheets in xlsx file for use in Tableau
with pd.ExcelWriter("beers_breweries2022.xlsx", engine = 'xlsxwriter') as writer:
    df_brewery.to_excel(writer, sheet_name="breweries")  
    df_beers.to_excel(writer, sheet_name="beers") 


ValueError: Length of values (5538) does not match length of index (5539)